In [ ]:
# following tutorial : https://starknetpy.readthedocs.io/en/latest/guide.html

In [ ]:
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.account.account import Account
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import KeyPair
import json
import os
from starknet_py.contract import Contract
from starknet_py.cairo.felt import encode_shortstring
from starknet_py.common import create_sierra_compiled_contract

from starknet_py.common import create_casm_class
from starknet_py.hash.casm_class_hash import compute_casm_class_hash
from starknet_py.net.client_models import ResourceBounds, EstimatedFee, PriceUnit
from pprint import pprint

In [ ]:
NODE_URL = "https://free-rpc.nethermind.io/sepolia-juno"
ACCOUNTS_PATH = os.path.join("..", "sepolia.json")

CONTRACT_DIR = os.path.join("..", "contracts")
CONTRACT_COMPILED_PATH = os.path.join(CONTRACT_DIR, "oracle_consensus_OracleConsensusNDS.compiled_contract_class.json")
CONTRACT_CLASS_PATH = os.path.join(CONTRACT_DIR, "oracle_consensus_OracleConsensusNDS.contract_class.json")

In [ ]:
DECLARED_CONTRACT = "0x366c68f0d32b432d120514b1daf9ba6fcc797420747ec0fb1bd991e3413326f"
DEPLOYED_CONTRACT = "0x2c6df17244002bbdb52e38bfed2216a13180aefffae98e7a33c12c79800a605"
#"0x75cc5abc0742e260c282b489625a3b7d9111c9fc230e74b9183025ecfc09ef5"


In [ ]:
client = FullNodeClient(node_url=NODE_URL)

In [ ]:
def address_to_felt(x : str) -> int :
    return int(x, 16)

## Connect accounts
---

In [ ]:
def retrieve_accounts():
    with open(ACCOUNTS_PATH, 'r') as file:
        data = json.load(file)
    
    addresses = data['addresses']
    keys = data['private_keys']

    return addresses, keys

accounts_addresses, accounts_keys = retrieve_accounts()

In [ ]:
accounts = [
    Account(
        client=client,
        address=address,
        key_pair=KeyPair.from_private_key(key),
        chain=StarknetChainId.SEPOLIA
    ) for address, key in zip(accounts_addresses, accounts_keys)
]

len(accounts)

## Declare / Deploy Contract

---

In [ ]:
def get_built_contract() -> dict :
    result = dict()

    with open(CONTRACT_COMPILED_PATH, 'r', encoding='utf-8') as file:
        result['compiled'] = file.read()

    with open(CONTRACT_CLASS_PATH, 'r', encoding='utf-8') as file:
        result['class_hash'] = file.read()

    result['abi'] = create_sierra_compiled_contract(
        compiled_contract=result['class_hash']
    ).parsed_abi

    return result

abi = get_built_contract()['abi']

In [ ]:
async def declare():
    account = accounts[0]

    contract_file_content = get_built_contract()
    
    casm_class = create_casm_class(contract_file_content['class_hash'])
    compiled_class_hash = compute_casm_class_hash(casm_class)

    declare_transaction = await account.sign_declare_v3(
        contract_file_content['compiled'], compiled_class_hash
    )

    resp = await account.client.declare(transaction=declare_transaction)
    await account.client.wait_for_tx(resp.transaction_hash)

    print(f"declared at : {resp.class_hash}")

    return resp.class_hash

async def deploy(abi, class_hash):

    constructor_args = {
        "admins" : [ int(x, 16) for x in
            accounts_addresses[:3]],

        "enable_oracle_replacement" : True,
        "required_majority" : 2,
        "n_failing_oracles" : 2, 
        
        "constrained" : True,
        "unconstrained_max_spread" : 0,
        "dimension" : 2,
        "oracles": [
            int(x, 16) for x in accounts_addresses
        ],
    }

    deploy_result = await Contract.deploy_contract_v3(
        account=accounts[2],
        class_hash=class_hash,
        abi=abi,
        l1_resource_bounds=ResourceBounds(400000, 99925789148178),#0.000000015233456997),
        constructor_args=constructor_args
    )

    await deploy_result.wait_for_acceptance()

    return deploy_result.deployed_contract()

In [ ]:
# declared_contract = await declare()
# declared_contract

In [ ]:
# contract = await deploy(abi, DECLARED_CONTRACT)
# contract
# 0x041a78e741e5af2fec34b695679bc6891742439f7afb8484ecd7766661ad02bf

## Use the Existing contract

---

In [ ]:
def replace_i128_with_u128(data):
    if isinstance(data, dict):
        return {key: replace_i128_with_u128(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [replace_i128_with_u128(item) for item in data]
    elif isinstance(data, str):
        return data.replace('i128', 'u128')
    else:
        return data

# abi_as_u128 = replace_i128_with_u128(abi)

def to_hex(x: int) -> str:
    return f"0x{x:0x}"

In [ ]:
# contract = Contract(address=DEPLOYED_CONTRACT, abi=abi, provider=accounts[0])
# contract = await Contract.from_address(provider=accounts[0], address=DEPLOYED_CONTRACT)

In [ ]:
# resb = ResourceBounds(400000, 99925789148178)
# contract.functions['update_prediction'].abi
# contract.functions

In [ ]:
predictions = [
    [492954726014948928, 334814622544049920],
    [437692571090454848, 410445303499263744], 
    [967794129545080320, 564219801545577856], 
    [431029386438835904, 387225378439864320], 
    [487609527760088832, 337990045876390976], 
    [284178293988284864, 485072442019714880], 
    [990059578132686080, 558600821433541504]
]

In [ ]:
async def update_all_the_predictions():
    for i, (prediction, account) in enumerate(zip(predictions, accounts)):
        # if i < 5: continue
        
        print(f"- {i} : {prediction}")
        print("- add: ",to_hex(account.address))

        contract = await Contract.from_address(
            provider=account, address=DEPLOYED_CONTRACT
        )

        print(to_hex(contract.address))

        r = ResourceBounds(400000, 40932837875699)
        f = contract.functions['update_prediction'].prepare_invoke_v3(
            prediction=predictions[i], l1_resource_bounds=r
        )

        print(await f.estimate_fee())

        result = await f.invoke()
        print(result)
        print("------------------")
    
await update_all_the_predictions()

In [ ]:
        # consensus_value: LegacyMap<usize, i128>, // wad convention
        # consensus_reliability_second_pass : i128, // wad convention
        # consensus_reliability_first_pass : i128 // wad convention

In [ ]:
contract = await Contract.from_address(
    provider=accounts[0], address=DEPLOYED_CONTRACT
)

In [ ]:
def wad_to_float(x) :
    return float(x) * 1e-18

In [ ]:
contract.functions

In [ ]:
prediction = await contract.functions['get_consensus_value'].call()
first_pass_consensus_reliability = await contract.functions["get_first_pass_consensus_reliability"].call()
second_pass_consens_reliability = await contract.functions["get_second_pass_consensus_reliability"].call()

In [ ]:
print([wad_to_float(x) for x in prediction[0]])
print(wad_to_float(first_pass_consensus_reliability[0]))
print(wad_to_float(second_pass_consens_reliability[0]))